# PROJECT:

In [1]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res

In [2]:
df = pd.read_table('lpmc11.dat')
database = db.Database('lpmc', df)
df

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,1,0,0,1,4,3,1,1,1.0,1,...,0.109444,0.000000,0.055556,0.000000,0,0.059444,1.5,0.15,0.0,0.112150
1,13,1,1,1,4,3,1,5,0.0,1,...,0.241389,0.000000,0.122222,0.000000,0,0.132222,0.0,0.50,0.0,0.065126
2,19,4,0,1,3,3,6,5,0.0,1,...,0.222500,0.000000,0.312222,0.000000,0,0.221667,0.0,0.56,0.0,0.086466
3,20,5,1,0,4,3,1,5,0.0,1,...,0.381667,0.000000,0.062222,0.000000,0,0.117222,0.0,0.41,0.0,0.097156
4,39,9,2,0,4,3,1,5,0.0,1,...,0.146944,0.000000,0.225000,0.000000,0,0.200833,0.0,0.48,0.0,0.378976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,81035,17603,1,1,3,1,1,1,1.0,3,...,0.278056,0.216667,0.000000,0.000000,0,0.392222,2.4,0.98,0.0,0.547450
4996,81040,17604,2,0,3,1,1,2,0.0,3,...,0.264444,0.000000,0.353333,0.176667,1,0.288889,0.0,0.85,0.0,0.175000
4997,81045,17605,0,1,4,3,1,5,0.0,3,...,0.128889,0.000000,0.045833,0.000000,0,0.067778,0.0,0.17,0.0,0.024590
4998,81066,17608,0,3,3,3,6,1,1.0,3,...,0.092222,0.000000,0.389444,0.000000,0,0.193333,1.5,0.61,0.0,0.485632


# Model 0

**Variables:**

In [3]:
dur_walking = Variable('dur_walking') #time for walking
dur_cycling = Variable('dur_cycling') #time for cycling

dur_pt_access = Variable('dur_pt_access')
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int')

dur_pt = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total time for public transport
dur_driving = Variable('dur_driving')

cost_transit = Variable('cost_transit') #cost for public transport
cost_driving_fuel = Variable('cost_driving_fuel')
cost_driving_ccharge = Variable('cost_driving_ccharge')

cost_driving = cost_driving_fuel + cost_driving_ccharge #total cost for driving

travel_mode = Variable('travel_mode') #Choice

**Parameters:**

In [4]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

In [9]:
v_walk_Model0 = (
    constant_walking
    + beta_time * dur_walking
)

v_cycling_Model0 = (
    constant_cycling
    + beta_time * dur_cycling
)

v_PT_Model0 = (
    constant_pt
    + beta_cost * cost_transit
    + beta_time * dur_pt
)

v_driving_Model0 = (
    beta_cost * cost_driving
    + beta_time * dur_driving
)

In [10]:
V_Model0 = {1: v_walk_Model0, 2: v_cycling_Model0, 3: v_PT_Model0, 4: v_driving_Model0}
logprob_Model0 = models.loglogit(V_Model0, None, travel_mode)
biogeme_Model0 = bio.BIOGEME(database, logprob_Model0) #create the biogeme estimator
biogeme_Model0.modelName = 'logit_Model0'
res_Model0 = biogeme_Model0.estimate()

In [11]:
print(res_Model0.printGeneralStatistics())

Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4676.875
Final log likelihood:	-4676.875
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00107
Akaike Information Criterion:	9363.751
Bayesian Information Criterion:	9396.337
Final gradient norm:	2.6771E-04
Nbr of threads:	8



In [12]:
res_Model0.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.156585,0.012667,-12.361233,0.0
beta_time,-5.226955,0.191905,-27.237207,0.0
constant_cycling,-2.485329,0.086252,-28.814839,0.0
constant_pt,0.726783,0.047157,15.411971,0.0
constant_walking,1.187012,0.078799,15.063738,0.0


The sign of the 2 generic coefficent leaves us satisfied. We know it's reasonable that utility is decreased when cost and/or travel time increase. It also seems that time impacts more the utility, since in asbolute value beta_time is higher than beta_cost. Concernig the ASCs, we normalized with respect to ASC_car so this is our baseline. We notice that ASC_walking is the highest so it seems that people, assuiming the same atrributes values, would choose this alternative. Anyway we see that, since the value of beta_time is high in absoulute value, and walking usually requires more time than the others, it will be penalized, as we expected. Cycling seems really disadvantaged, since its ASC is already low, and it has the same beta_time (due to generic coefficent assumption). Significance of value is confirmed by the t-test values that are all above the 1.96 threshold

# Model 1

We decide to split beta_time into 4 alternative specific parameters. Our choice is due to the fact that this coefficent is present in every utility function, and we think it can undergo a wider variation, since time perception is strictly related to the chosen travel mode.

In [13]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)

In [14]:
v_walk_Model1 = (
    constant_walking
    + beta_time_walking * dur_walking
)

v_cycling_Model1 = (
    constant_cycling
    + beta_time_cycling * dur_cycling
)

v_PT_Model1 = (
    constant_pt
    + beta_cost * cost_transit
    + beta_time_pt * dur_pt
)

v_driving_Model1 = (
    beta_cost * cost_driving
    + beta_time_driving * dur_driving
)

In [15]:
V_Model1 = {1: v_walk_Model1, 2: v_cycling_Model1, 3: v_PT_Model1, 4: v_driving_Model1}
logprob_Model1 = models.loglogit(V_Model1, None, travel_mode)
biogeme_Model1 = bio.BIOGEME(database, logprob_Model1)
biogeme_Model1.modelName = 'logit_Model1'
res_Model1 = biogeme_Model1.estimate()

In [16]:
print(res_Model1.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-6931.472
Final log likelihood:	-4372.911
Likelihood ratio test for the init. model:	5117.122
Rho-square for the init. model:	0.369
Rho-square-bar for the init. model:	0.368
Akaike Information Criterion:	8761.822
Bayesian Information Criterion:	8813.959
Final gradient norm:	2.8918E-03
Nbr of threads:	8



In [17]:
res_Model1.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.136269,0.013923,-9.787478,0.000000e+00
beta_time_cycling,-5.436177,0.492925,-11.028412,0.000000e+00
beta_time_driving,-5.794675,0.384636,-15.065358,0.000000e+00
beta_time_pt,-3.127377,0.243891,-12.822864,0.000000e+00
beta_time_walking,-8.161128,0.386290,-21.126960,0.000000e+00
constant_cycling,-2.560688,0.150700,-16.991966,0.000000e+00
constant_pt,-0.405922,0.069445,-5.845207,5.059401e-09
constant_walking,1.933359,0.129143,14.970653,0.000000e+00


As before, signs are reasonable. Concernign the values of the coefficents, we firstly notice that beta_cost reamined pretty much the same and it seems reasonable becuase people usually care how much money they spend rather than for what they do spend. The wide range of values we can see on beta_time strenghtens our initial assumption, in fact we see how beta_walking is really low, since walking requires phisycal effort, while the two alternatives that just require the individual's focus (driving and cycling) are pretty similar and this seems fine. In the end we see that beta_time_PT is instead the lowest in absolute value, and this is logic for us since when you are in public transport you are able to do other activities or just realx without having to mantain focus. Again, as before, ASC-walking is still bigger than car, as cycling is still the lowest. The only change we notice is the sign of ASC_pt, which is probably due to the value of the beta_time_pt we previously analyzed. Significance of value is confirmed by the t-test values that are all above the 1.96 threshold

In [18]:
res_Model1.likelihood_ratio_test(res_Model0, 0.01)

LRTuple(message='H0 can be rejected at level 1.0%', statistic=607.9289925585272, threshold=11.344866730144373)

To compare the 2 models we used the likelihood ratio test. This is doable since model 0 is a linear restriction of model 1 (assuming all the alternative specific beta_time equal). Our null-hypotesis is that the resricted model (in this case model 0) is the true model. We set the significance level of the test to 0,01 and as we can see from the result the null-hypotesis is rejected, hence our preferred model is model 1

# Model 2:

- Interaction of age with ASCs:

We select 'age' as socio-economic characteristic since we think it has a stornger correlation with the choice of a transport mode rather than sex, which is the other socio-ecomic characteristci we found in the dataset. In fact, if we assume two individuals have the same age, from our experience, their choice (putting all the other parametres equal) would not probably be different. On the other hand, age can impact the choice both from an availability point of view (i.e. young or really old may not have a driving license) and a physical effort one.

In [22]:
age = Variable('age')

In [50]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
ASC_w = Beta('ASC_w', 0, None, None, 0)
ASC_c = Beta('ASC_c', 0, None, None, 0)
ASC_pt = Beta('ASC_pt', 0, None, None, 0)


In [56]:
interacted_ASC_w = ASC_w * age/99
interacted_ASC_c = ASC_c * age/99
interacted_ASC_pt = ASC_pt * age/99


In [57]:
v_w_Model2_ASC = (
    interacted_ASC_w
    + beta_time_walking * dur_walking
)

v_c_Model2_ASC = (
    interacted_ASC_c
    + beta_time_cycling * dur_cycling
)

v_pt_Model2_ASC = (
    interacted_ASC_pt
    + beta_cost * cost_transit
    + beta_time_pt * dur_pt
)

v_driving_Model2_ASC = (
    beta_cost * cost_driving
    + beta_time_driving * dur_driving
)

In [58]:
V_Model2_ASC = {1: v_w_Model2_ASC , 2: v_c_Model2_ASC, 3: v_pt_Model2_ASC, 4: v_driving_Model2_ASC}
logprob_Model2_ASC = models.loglogit(V_Model2_ASC, None, travel_mode)
biogeme_Model2_ASC = bio.BIOGEME(database, logprob_Model2_ASC)
biogeme_Model2_ASC.modelName = 'logit_Model2_ASC'
res_Model2_ASC = biogeme_Model2_ASC.estimate()

In [59]:
print(res_Model2_ASC.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-5340.819
Final log likelihood:	-4615.576
Likelihood ratio test for the init. model:	1450.488
Rho-square for the init. model:	0.136
Rho-square-bar for the init. model:	0.134
Akaike Information Criterion:	9247.151
Bayesian Information Criterion:	9299.289
Final gradient norm:	1.8211E-02
Nbr of threads:	8



In [60]:
res_Model2_ASC.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_c,-4.876922,0.374369,-13.027038,0.000000e+00
ASC_pt,-0.998334,0.133145,-7.498069,6.483702e-14
ASC_w,1.531922,0.158594,9.659391,0.000000e+00
beta_cost,-0.150730,0.014565,-10.348483,0.000000e+00
beta_time_cycling,-8.106052,0.665896,-12.173145,0.000000e+00
beta_time_driving,-5.752060,0.391634,-14.687323,0.000000e+00
beta_time_pt,-3.124069,0.244505,-12.777127,0.000000e+00
beta_time_walking,-5.265453,0.197672,-26.637256,0.000000e+00


Need to find a test! (Probably a Cox test would be ok)

When we have to analyze the interaction of the age with one of the variables, we decided to link it with cost, since we think that indviduals may have a differtent perception of money depending on their age. For example, when you're a student you give a different value to money than the one you give them when you've been working for several years.

- Interaction of age with time:

In [61]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)

In [62]:
beta_cost_interacted = beta_cost * age/99

In [63]:
v_w_Model2_cost = (
    constant_walking
    + beta_time_walking * dur_walking
)

v_c_Model2_cost = (
    constant_cycling
    + beta_time_cycling * dur_cycling
)

v_pt_Model2_cost= (
    constant_pt
    + beta_cost_interacted * cost_transit    
    + beta_time_pt * dur_pt
)

v_driving_Model2_cost = (
    beta_cost_interacted * cost_driving
    + beta_time_driving * dur_driving
)

In [64]:
V_Model2_cost = {1: v_w_Model2_cost, 2: v_c_Model2_cost, 3: v_pt_Model2_cost, 4: v_driving_Model2_cost}
logprob_Model2_cost = models.loglogit(V_Model2_cost, None, travel_mode)
biogeme_Model2_cost = bio.BIOGEME(database, logprob_Model2_cost)
biogeme_Model2_cost.modelName = 'logit_Model2_cost'
res_Model2_cost = biogeme_Model2_cost.estimate()

In [65]:
print(res_Model2_cost.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4443.281
Final log likelihood:	-4387.311
Likelihood ratio test for the init. model:	111.9388
Rho-square for the init. model:	0.0126
Rho-square-bar for the init. model:	0.0108
Akaike Information Criterion:	8790.622
Bayesian Information Criterion:	8842.76
Final gradient norm:	1.4149E-06
Nbr of threads:	8



In [66]:
res_Model2_cost.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.287265,0.039548,-7.263774,3.763656e-13
beta_time_cycling,-5.430248,0.494069,-10.990875,0.000000e+00
beta_time_driving,-5.958972,0.382057,-15.597075,0.000000e+00
beta_time_pt,-3.218753,0.242768,-13.258575,0.000000e+00
beta_time_walking,-8.180628,0.386822,-21.148285,0.000000e+00
constant_cycling,-2.566973,0.150922,-17.008642,0.000000e+00
constant_pt,-0.407455,0.069331,-5.876924,4.179601e-09
constant_walking,1.939273,0.129411,14.985358,0.000000e+00


In [78]:
v_w_cox1 = v_walk_Model1 + v_w_Model2_ASC
v_c_cox1 = v_cycling_Model1 + v_c_Model2_ASC
v_pt_cox1 = v_PT_Model1 + v_pt_Model2_ASC
v_driving_cox1 = v_driving_Model1 + v_driving_Model2_ASC

In [79]:
V = {1: v_w_cox1, 2: v_c_cox1, 3: v_pt_cox1, 4: v_driving_cox1}
logprob_cox1 = models.loglogit(V, None, travel_mode)
biogeme_cox1 = bio.BIOGEME(database, logprob_cox1)
biogeme_cox1.modelName = 'logit_cox1'
res_cox1 = biogeme_cox1.estimate()


In [96]:
print(res_cox1.likelihood_ratio_test(res_Model1, 0.1))
#print(res_Model2_ASC.likelihood_ratio_test(res_cox1, 0.1))
print(res_cox1.likelihood_ratio_test(res_Model2_ASC, 0.1))

LRTuple(message='H0 can be rejected at level 10.0%', statistic=49.127263585955006, threshold=6.251388631170325)
LRTuple(message='H0 can be rejected at level 10.0%', statistic=534.4566875202636, threshold=6.251388631170325)


In [88]:
v_w_cox2 = v_walk_Model1  
v_c_cox2 = v_cycling_Model1
v_pt_cox2 = v_PT_Model1 + beta_cost_interacted * cost_transit
v_driving_cox2 = v_driving_Model1 + beta_cost_interacted * cost_driving

In [89]:
V = {1: v_w_cox2, 2: v_c_cox2, 3: v_pt_cox2, 4: v_driving_cox2}
logprob_cox2 = models.loglogit(V, None, travel_mode)
biogeme_cox2 = bio.BIOGEME(database, logprob_cox2)
biogeme_cox2.modelName = 'logit_cox2'
res_cox2 = biogeme_cox2.estimate()

In [98]:
print(res_cox2.likelihood_ratio_test(res_Model1, 0.01))
print(res_cox2.likelihood_ratio_test(res_Model2_cost, 0.01))

LRTuple(message='H0 can be rejected at level 1.0%', statistic=5.578423499724522, threshold=nan)


BiogemeError: The unrestricted model (-4387.311206863418, 8) has a lower log likelihood than the restricted one (-4375.700002695043, 8)